# Работа с базой данных для модели микроуровневого стресс-тестирования (банкротства застройщиков)

<div class="alert alert-block alert-warning">
<b>Этот блокнот может вызываться из другого блокнота с параметрами - путем к файлу slite3 с базой данных для модели "банкротств застройщиков"<br>
Проверяйте, с какой именно базой работаете</b>
</div>

***
<p>Блокнот предназначен для создания и обновления базы данных для модели "банкротства застройщиков"</p>

**Источники данных:**
  - xlsx-файл с данными, передаваемый ДОМ-РФ (NOZA)
  - xlsx-файл с данными, скачиваемый с платной базы данных SPARK
  - http://cbr.ru/statistics/bank_sector/int_rat/ xlsx-файл с данными "Сведения по кредитам в рублях, долларах США и евро нефинансовым организациям", используется для моделирования показателя Z_A (Чистая кредиторская задолженность (по модулю), медиана по отрасли)
  
<p>Данные поставляются в виде файлов, стандартно размещаются в папке ./DB/SRC (кроме файла с кредитной ставкой, он обрабатывается непосредствено с сайта ЦБР)</p>
<div class="alert alert-block alert-warning">
<b>Блокнот настроен на определенный формат обоих файлов, при изменения формата необходима коррекция кода</b>
</div>

***

#### Алгоритм работы

  - считываем данные из Ексель-файлов
  - выбираем нужные поля, чистим данные, переименовываем поля для удобного использования в коде
  - трансформируем форму представления данных в удобный для Питона формат
  - назначаем ключевые поля - для таблицы noza - поле **Идентификационный номер объекта**, для таблицы SPARK ключ будет составным из полей **ИНН** и **года**
  - записываем обе таблицы в базу данных (по умолчанию - <b>./DB/bankrp.sqlite3</b>). <b>В две разные таблицы - noza и spark</b> В дальнейшем модель работает именно с этим файлом и таблицами. Тип ключевых полей:
    - ИД объекта (имя в базе **obj_id**) - **int**
    - ИНН (имя в базе - **inn**) - **string**
    - Год (имя в базе - **year**) - **int**

In [45]:
%%javascript

// В этой ячейке читаем параметры вызова данного блокнота - ожидается имя рабочей базы данных 
// (отличное от имени по умолчанию). Код этой ячейки - java-script

IPython.notebook.kernel.execute("URL = '" + window.location + "'");

<IPython.core.display.Javascript object>

In [46]:
from os import path, remove
import urllib.parse as urlparse
from urllib.parse import parse_qs, urljoin
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
import sqlalchemy as sa
import locale
from cmasf import pandas_sql as pds
from IPython.display import Markdown as md

#### Устанавливаем значения общих переменных (далее константы)

In [47]:
# пути к файлам
strBasePath=path.join('DB', 'bankrp.sqlite3') # путь к рабочей базе данных SQLite - может быть изменено в вызывающем блокноте
strNOZA_XLSX=path.join('DB', 'SRC', 'noza2020.xlsx') # пусть к файлу с исходными даными для BD (NOZA)
strNOZA_sheet='Строящиеся дома'
strSPARK_XLSX=path.join('DB', 'SRC', 'SPARK.xlsx') # пусть к файлу с исходными даными для BD (SPARK)
strNOZA_inn=path.join('DB', 'Temp', 'inn_{date}.csv') # пусть к файлу с уникальными ИНН из BD NOZA для запроса BD SPARK
# ------------

# константы базы данных SQLite3
strNOZA_table='noza' # название таблицы NOZA в базе данных SQLite
strNOZA_data_pass='noza_columns' # таблица с названиями колонок для таблицы noza
strSPARK_table='spark' # название таблицы SPARK в базе данных SQLite
strSPARK_data_pass='spark_columns' # таблица с названиями колонок для таблицы SPARK
strInfoTalbe='update_info' # таблица с информацией об обновлении базы даных
# ------------

# константы для NOZA

# поля базы данных NOZA, используемые для хранения и расчетов (остальные поля импортироваться в базу не будут)
iNOZAFileType=2

if iNOZAFileType==1:
    lstNOZA_use_cols=['ИНН застройщика',
     'Идентификационный номер объекта', # ключевое поле, уникальное
     'Наименование застройщика',
     'Организационно-правовая форма',
     'Размер уставного капитала',
     'Субъект Российской Федерации (регион)',
     'Проектная площадь жилых помещений',
     'Минимальная цена 1 кв м всех квартир без учета скидок',
     'Минимальная цена проектной площади жилых помещений, руб.',
     'Запланированный срок ввода в эксплуатацию (текущий)',
     'Планируемая стоимость строительства',
     'Новая планируемая стоимость строительства']

    # список новых имен для используемых полей базы NOZA (порядок соотв. предыдущему списку)
    lst_NOZA_names=['inn', 'obj_id', 'Name', 'OPF', 'Capital', 'REGION', 'Sq_living_proj', 'Min_proce_1sqm', 
               'Min_price_living_r', 'Input_date', 'Bld_price_proj', 'Bld_price_proj_new']
    strNOZAKey='obj_id'
else:
    
    lstNOZA_use_cols= ['Отчетная дата',
     'Идентификатор дома',
     'ИНН юрлица',
     'Наименование юрлица',
     'Регион',
     'Метод обеспечения ответственности',
     'Стоимость строительства,\nруб.',
     'Жилая площадь,\nкв. м',
     'Планируемая дата ввода']
    lst_NOZA_names=['pub_date', 'obj_id', 'inn', 'Name', 'REGION', 'Resp', 'bld_price', 'sq_living', 'Input_date']
    strNOZAKey=['pub_date', 'obj_id']



#константы для SPARK
# поля базы данных SPARK, используемые для хранения и расчетов 
# (кроме этих полей выбираются так же все поля, имеющие 4 цифры года в заголовке)
lstSPARK_use_cols=['Наименование',
 'Дата ликвидации',
 'Код налогоплательщика',
 'Регион регистрации']

# список новых имен для используемых полей базы SPARK (порядок соотв. предыдущему списку)
lst_SPARK_names=['Name', 'Cancel_date', 'inn', 'REGION']

# словарь переименования полей базы данных SPARK, содержащих год в заголовке (цифры года уйдут в новое поле индекса)
dctDFS={'receivables':'Дебиторская задолженность, RUB', 'actives':'Активы  всего, RUB', 
            'capital':'Уставный капитал , RUB', 'acc_pay':'Кредиторская задолженность, RUB', 
            'profit':r'Чистая прибыль \(убыток\), RUB', 'roa':r'Рентабельность активов \(ROA\), %'}

lstSPARKIndexFields=['year', 'inn'] # список ключевых полей для SPARK
#---------------

#константы для LOAN
strLOAN_table='loan' # название таблицы LOAN в базе данных SQLite
strLOAN_data_pass='loan_columns' # таблица с названиями колонок для таблицы loan

In [48]:
# блок для парсинга параметров вызова данного блокнота 
# изменяет рабочую базу данных по умолчанию на указанную в параметрах вызова
parsed = urlparse.urlparse(URL)
try:
    strBasePath=parse_qs(parsed.query)['DATABASE'][0] 
except KeyError:
    pass

strMess='''<hr><div class="alert alert-block alert-info">
<b>Рабочая база данных - {file_name}</b></div>  <hr>  '''.format(file_name=strBasePath)

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strBasePath)) # connection к рабочей базе данных
md(strMess)

<hr><div class="alert alert-block alert-info">
<b>Рабочая база данных - DB/bankrp.sqlite3</b></div>  <hr>  

создание (если не существует) дополнительной таблицы с информацией об обновлении базы данных

In [49]:
strCreateQ='''
CREATE TABLE IF NOT EXISTS {tab_name} ("utable" TEXT, "udate" TEXT NOT NULL, PRIMARY KEY("utable"))
'''

conWork.execute(strCreateQ.format(tab_name=strInfoTalbe))
print('таблица создана')

таблица создана


### Обработка базы данных NOZA

<p>Алгоритм обработки:</p>

  - читаем таблицу из файла Ексель (`strNOZA_XLSX`, преполагается, что там один лист с нужной таблицей - первый)
  - выбираем нужные поля, переименовываем их в удобный формат
  - из поля `Запланированный срок ввода в эксплуатацию (текущий)` (`Input_date`) выбираем год (`year`) (предварительно поле преобразуется в тип `datetime`), создаем новое поле `year` типа `int`
  - поле `ИНН застройщика` (`inn`) преобразуется в тип `string`)
  - поле `Идентификационный номер объекта` (`obj_id`) преобразуется в тип `int`)
  - поля `'Размер уставного капитала', 'Проектная площадь жилых помещений', 'Минимальная цена 1 кв м всех квартир без учета скидок', 'Минимальная цена проектной площади жилых помещений, руб.', 'Планируемая стоимость строительства', 'Новая планируемая стоимость строительства'`(`'Capital', 'Sq_living_proj', 'Min_proce_1sqm', Min_price_living_r', 'Bld_price_proj', 'Bld_price_proj_new'`)  преобразуется в тип `float`)
  - удаляются записи с пустыми полями `ИНН застройщика` (`inn`) и `Запланированный срок ввода в эксплуатацию (текущий)` (`Input_date`)
  - в качестве индексов устанавливается поле `obj_id` и `year`.
  - полученный `DataFrame` записываем в таблицу `strNOZA_table` базы данных `strBasePath`
  
 <p>Дополнительно в ту же базу записываем словарь переименования полей из исходных в используемые в дальнейшем</p>

In [50]:
def NOZA_transform(pdf, file_type=iNOZAFileType):
    """функция очистки и преобразования таблицы XLS с данными NOZA в Dataframe"""

    pdfN=pdf.rename(columns=dict(zip(lstNOZA_use_cols, lst_NOZA_names))) # переименование колонок
    
    pdfN=pdfN[pdfN['inn'].notnull() & pdfN['Input_date'].notnull()] # убираем пустые ИНН и даты
    pdfN['completion_year']=pdfN['Input_date'].dt.year # выделяем из даты год запланированного ввода в эксплуатацию
    pdfN['completion_year']=pdfN['completion_year'].astype(int) 
    pdfN[strNOZAKey]=pdfN[strNOZAKey].astype(int) 
     
    if file_type==1:
        pdfN[['Capital', 'Sq_living_proj', 'Min_proce_1sqm', 
               'Min_price_living_r', 'Bld_price_proj', 'Bld_price_proj_new']]=pdfN[['Capital', 'Sq_living_proj', 'Min_proce_1sqm', 
               'Min_price_living_r', 'Bld_price_proj', 'Bld_price_proj_new']].apply(pd.to_numeric)
    else:
        pdfN[['bld_price', 'sq_living']]=pdfN[['bld_price', 'sq_living']].apply(pd.to_numeric)
        pdfN['pub_date']=pd.to_datetime(pdfN['pub_date'], format='%Y-%m-%d')
        pdfN=pdfN.loc[pdfN['pub_date']==pdfN.sort_values(by='pub_date', ascending=False)['pub_date'].iloc[0], :] # выбираем данные по последней отчетной дате
#         pdfN['pub_date']=pd.to_datetime(pdfN['pub_date'], format='%Y-%m-%d') 
    return pdfN.set_index(strNOZAKey)


# тип ИНН - string устанавливаем именно здесь, чтобы не возиться с лидирующими нулями
if iNOZAFileType==1:
    pdfNOZA=pd.read_excel(strNOZA_XLSX, header=3, usecols=lstNOZA_use_cols,  dtype={'ИНН застройщика':str}) # читаем данные из Excel
else:
    pdfNOZA=pd.read_excel(strNOZA_XLSX, sheet_name=strNOZA_sheet, usecols=lstNOZA_use_cols,  dtype={'ИНН юрлица':str}) # читаем данные из Excel
    
print('Прочитано из файла Excel - ', pdfNOZA.shape)
# print(pdfNOZA)
pdfNOZA=NOZA_transform(pdfNOZA)
print('После очистки - ', pdfNOZA.shape)
print('='*50)
print(pdfNOZA.head())

Прочитано из файла Excel -  (111706, 9)
После очистки -  (9614, 8)
                          inn                             Name  \
pub_date   obj_id                                                
2020-03-26 2230    5321078745                      ПРОЕКТСТРОЙ   
           2569    4632005459  МКП УКС АДМИНИСТРАЦИИ Г. КУРСКА   
           4796    2311231205                 НЕМЕЦКАЯ ДЕРЕВНЯ   
           4892    2311231205                 НЕМЕЦКАЯ ДЕРЕВНЯ   
           29059   7448046410                      СЗ 10-Й ДОМ   

                                 REGION  \
pub_date   obj_id                         
2020-03-26 2230    Новгородская область   
           2569         Курская область   
           4796      Краснодарский край   
           4892      Краснодарский край   
           29059    Челябинская область   

                                                                Resp  \
pub_date   obj_id                                                      
2020-03-26 2230    Соотв

#### Запись подготовленного фрейма в рабочую базу

*Используется доработанный класс `DataFrame` - `DataFrameDATA` для возможности полноценного использования `UPSERT` - добавления отсутсвующих записей и обновления существующих* 

In [51]:
_pdf=pds.DataFrameDATA(pdfNOZA)

_pdf.to_sql(strNOZA_table, con=conWork, if_exists='upsert', chunksize=int(1e4))

print('Done writing to SQLite3 {} to table {}'.format(strBasePath, strNOZA_table), _pdf.shape)
_ser=pd.Series(dict(zip(lst_NOZA_names, lstNOZA_use_cols)), name='col_name')
_ser.at['completion_year']= 'Год планируемого ввода'
_ser.to_sql(strNOZA_data_pass, con=conWork, if_exists='replace')

print('Done writing NOZA columns names to SQLite3 {} to table {}'.format(strBasePath, strNOZA_data_pass))

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTalbe, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), table_name=strNOZA_table))

Done writing to SQLite3 DB/bankrp.sqlite3 to table noza (9614, 8)
Done writing NOZA columns names to SQLite3 DB/bankrp.sqlite3 to table noza_columns


#### Подготовка запросов для таблицы NOZA
Так как имеется два формата данных NOZA (а, возможно, будет и больше) для каждого формата создается отдельный агрегирующий запрос.  
Запрос, построенных для другого типа исходного формата данных, в базе работать не будет

In [52]:
if iNOZAFileType==1:
    pass # пока не реализовано, основной формат = 2
elif iNOZAFileType==2:
    strNOZA_CreateAGGview='''
create view if not exists noza_ag as
select {noza_table}.pub_date, {noza_table}.inn, {noza_table}.completion_year, 
sum({noza_table}.bld_price) as price, sum({noza_table}.sq_living) as square
from {noza_table}
group by {noza_table}.pub_date, {noza_table}.inn, {noza_table}.completion_year;
    '''.format(noza_table=strNOZA_table)
else:
    pass
conWork.execute(strNOZA_CreateAGGview)

print('done create')

done create


#### Подготовка данных для запроса в базу данных SPARK

Из базы данных SPARK запрашиваем данные только для тех компаний, которые имеются в базе данных NOZA. Для запроса используем список ИНН компаний из NOZA (только уникальные).

Список сохраняем в csv (txt) файле, этот файл передаем на вход сервиса SPARK

In [53]:
strFileINN=strNOZA_inn.format(date=dt.datetime.now().strftime('%Y_%m_%d'))
# pdfNOZA.inn.unique().to_csv(strFileINN, sep=';', encoding='cp1251')
with open(strFileINN, 'w') as inn_file:
    for i in pdfNOZA.inn.unique():
        inn_file.write(i + '\n')

print('{cnt} ИНН записано в файл {name}'.format(cnt=len(pdfNOZA.inn.unique()), name= strFileINN))

3339 ИНН записано в файл DB/Temp/inn_2020_08_07.csv


***
***

### Обработка базы данных SPARK

<p>База SPARK представляет собой файл MS Excel, сформированный программой выдачи интернет-ресурса</p>
<p>Особенность формата: годовые данные выводятся в колонках, при этом имя колонки содержит номер года и название поля, то есть : </p>

| 2015, Активы  всего, RUB | 2016, Активы  всего, RUB |	2017, Активы  всего, RUB | ... |  
|--------------------------|--------------------------|--------------------------|-----|
| 13 222 000 | 103 491 000 |	126 160 000 | ... |

<p>Необходимо разделить названия и года, преобразовать из широкого представления данных в длинный, то есть к виду: <p>

| year | Активы  всего, RUB | ... |  
|------|----------------|-----|
| 2015 | 13 222 000     | ... |
|------|----------------|-----|
| 2016 | 103 491 000    | ... |
|------|----------------|-----|
| 2017 | 126 160 000    | ... |
    
<p>Таких полей 6: <i>'Дебиторская задолженность, RUB', 'Активы  всего, RUB', 'Уставный капитал , RUB', 'Кредиторская задолженность, RUB', 'Чистая прибыль (убыток), RUB', 'Рентабельность активов (ROA), %'</i></p>

   - После преобразования получившиеся колонки переименовываются в удобные для работы названия
   - Корректируются типы колонок (эти 6 полей должны иметь тип `float`), поле `ИНН` (`inn`) - `string`, поле `year` - `int`
   - Удяляются записи с пустыми значениями в полях `inn` и `year`
   - Для получившегося `DataFrame` устанавливаются в качестве индексов комбинация полей `inn` и `year`
   - Получившийся `DataFrame` записываем в таблицу `strSPARK_table` базы данных `strBasePath`

In [54]:
def SPARK_transform(pdf):
    """ подготовка базы данных СПАРК для использования в обучающей последовательности """
    
    dctRes=dict()
    
    # p_name=pdfSPARK[['INN', 'Name', 'REGION']].drop_duplicates().set_index('INN')

    _use_cols1=lstSPARK_use_cols+[c for c in pdf.columns if re.match(r'\d{4}', c)] # список: заданные колонки и колонки, начинающиеся с года
    pdfSrc=pdf[_use_cols1].rename(columns=dict(zip(lstSPARK_use_cols, lst_SPARK_names))) # переименовываем в новые названия для базы 

    for k, v in dctDFS.items():
        _use_cols=['inn',]+[c for c in pdf.columns if re.search(v, c)] # список: ИНН и колонки, начинающиеся с года
        _pdf=pdfSrc[pdfSrc['inn'].notnull()] # новый ДатаФрейм, очищенный от пропусков ИНН
        _pdf=_pdf[_use_cols].set_index('inn') # ДатаФрейм с ИНН и одним показателем по нескольким годам
        _pdf.columns=_pdf.columns.str[:4].astype(int)
        _pdf=_pdf.unstack()
        _pdf.index.levels[0].astype(int)
        _pdf.name=k
        dctRes.setdefault(k, _pdf)
        
    pdfRes=pd.DataFrame(dctRes).join(pdfSrc[lst_SPARK_names].set_index('inn'), on='inn')
    pdfRes.index.names=lstSPARKIndexFields
    return pdfRes

# для заполнения СПАРК ист-данными - раскомментировать следующее
# strSPARK_hist1='СПАРК_Выборка_компаний_20200723_1612.xlsx'
# strSPARK_hist2='СПАРК_Выборка_компаний_20200723_1609.xlsx'
# strSPARK_hist3='СПАРК_Выборка_компаний_20200723_1614.xlsx' # !!! глючит!! дупликаты в базе !!!

# strSP_path=path.join('DB', 'SRC', strSPARK_hist2) # для заполнения базы СПАРК данными предшествующих годов (база позволяет выкачивать по 5 лет за один раз)
# pdfSPARK=pd.read_excel(strSP_path, header=3, dtype={'Код налогоплательщика':str})
#----------------------------------------

# для заполнения последней точки - раскомментировать следующее
pdfSPARK=pd.read_excel(strSPARK_XLSX, header=3, dtype={'Код налогоплательщика':str})
#---------------------------------------

print('Прочитано из файла Excel - ', pdfSPARK.shape)
print(pdfSPARK.shape)
# pdfSPARK[pdfSPARK['Код налогоплательщика'].duplicated(keep=False)].to_csv('dupls.csv', sep=';')
pdfSPARK=SPARK_transform(pdfSPARK)
print('После очистки - ', pdfSPARK.shape)
print('='*50)
print(pdfSPARK.head())

Прочитано из файла Excel -  (8674, 39)
(8674, 39)
После очистки -  (43440, 9)
                 receivables      actives    capital      acc_pay     profit  \
year inn                                                                       
2015 3906245291          NaN          NaN        NaN          NaN        NaN   
     3801132195    3414000.0   13222000.0        NaN    1656000.0   516000.0   
     5050102110  139989000.0  973110000.0    10000.0  512847000.0  1556000.0   
     7448046410          NaN      10000.0        NaN          NaN        NaN   
     1840032987   17597000.0   62803000.0  5050000.0   46430000.0    34000.0   

                    roa                                            Name  \
year inn                                                                  
2015 3906245291     NaN                     1 СТРОИТЕЛЬНЫЙ ХОЛДИНГ, ООО   
     3801132195  0.0781        1-ДСК, ООО СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК   
     5050102110  0.0020                                 10 КВ

#### Запись подготовленного фрейма в рабочую базу

*Используется доработанный класс `DataFrame` - `DataFrameDATA` для возможности полноценного использования `UPSERT` - добавления отсутсвующих записей и обновления существующих* 

In [55]:
_pdf=pds.DataFrameDATA(pdfSPARK.copy())
try:
    _pdf['Cancel_date']=_pdf['Cancel_date'].dt.strftime('%Y-%m-%d') # преобразование в строку чтобы убрать NaT (в SQLite все равно в строке лежит)
except AttributeError: # уже в строке
    pass
    
_pdf.to_sql(strSPARK_table, con=conWork, if_exists='upsert', chunksize=int(1e4))

print('Done writing to SQLite3 {} to table {}'.format(strBasePath, strSPARK_table), _pdf.shape)

dctSPARK_pass=dict(zip(lst_SPARK_names, lstSPARK_use_cols))
dctSPARK_pass.update({k:v.replace('\\', '') for k, v in dctDFS.items()})
pd.Series(dctSPARK_pass, name='col_name').to_sql(strSPARK_data_pass, con=conWork, if_exists='replace')

print('Done writing SPARK columns names to SQLite3 {} to table {}'.format(strBasePath, strSPARK_data_pass))

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTalbe, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), table_name=strSPARK_table))

Done writing to SQLite3 DB/bankrp.sqlite3 to table spark (43440, 9)
Done writing SPARK columns names to SQLite3 DB/bankrp.sqlite3 to table spark_columns


#### Подготовка запросов для таблицы SPARK
Данные модель будет получать посредством этого запроса

Готовим два варианта запроса:
  1. ROA компании используется из базы данных, не расчитывается; Z_A компании считается по формуле:
  
$Z\_A = |\frac{(КРЕДИТОРСКАЯ\_ЗАДОЛЖЕННОСТЬ - ДЕБИТОРСКАЯ\_ЗАДОЛЖЕННОСТЬ)}{АКТИВЫ\_ВСЕГО}|$
      
      
      
  2. ROA компании расчитывается по формуле: 
  
$ROA = ЧИСТАЯ\_ПРИБЫЛЬ - АКТИВЫ\_ВСЕГО$
      
      Z_A компании считается по формуле:
      
$Z\_A=|\dfrac{(КРЕДИТОРСКАЯ\_ЗАДОЛЖЕННОСТЬ - ДЕБИТОРСКАЯ\_ЗАДОЛЖЕННОСТЬ)}{АКТИВЫ\_ВСЕГО}|$

In [56]:
strSPARK_CreateAGGview='''
create view if not exists spark_ag as
select {spark_table}.inn, {spark_table}.year, {spark_table}.capital, {spark_table}.Cancel_date, 
{spark_table}.roa as ROA, 
abs( ({spark_table}.acc_pay-{spark_table}.receivables )/{spark_table}.actives) as Z_A
from {spark_table};
    '''.format(spark_table=strSPARK_table)

strSPARK_CreateAGGview_ROA_CALC='''
create view if not exists spark_ag_roa_calc as
select {spark_table}.inn, {spark_table}.year, {spark_table}.capital, {spark_table}.Cancel_date, 
({spark_table}.profit - {spark_table}.actives) as ROA, 
abs( ({spark_table}.acc_pay-{spark_table}.receivables )/{spark_table}.actives) as Z_A
from {spark_table};
    '''.format(spark_table=strSPARK_table)

conWork.execute(strSPARK_CreateAGGview)
conWork.execute(strSPARK_CreateAGGview_ROA_CALC)

print('done create')

done create


## Обработка базы данных кредитной ставки с сайта ЦБР

**Источник - http://cbr.ru/statistics/bank_sector/int_rat/**  
На странице источника ищем ссылки с ключевым текстом "Сведения по кредитам в рублях, долларах США и евро нефинансовым организациям"  

In [57]:
strCBR_LOAN_SRC=r'http://cbr.ru/statistics/bank_sector/int_rat/'

strKeyText='Сведения по кредитам в рублях, долларах США и евро нефинансовым организациям'.replace(' ', '\s+')

re_t=re.compile(strKeyText)

resp=requests.get(strCBR_LOAN_SRC)

loan_soup=BeautifulSoup(resp.text)

div_docs=loan_soup.findAll('div', class_='document-regular')

print('Найдено инфо-тегов', len(div_docs))

Найдено инфо-тегов 28


Читаем файл по сылке, искомая ставка в столбце `F` листа `ставки_руб`.  
Преобразуем даты в формат datetime

In [58]:
def find_link(div_tags):
    for d in div_tags:
        if d.find('span', class_='document-regular_name_visible', text=re_t) and d.find('div', class_='document-regular_comment', text=re.compile('в целом по Российской Федерации'.replace(' ', '\s'))):
            pub_date=dt.datetime.strptime(d.find('div', class_='document-regular_date').text.strip(), '%d.%m.%Y')
            a=d.find('a')
            
            print('Найден файл с нужными данныи, дата публикации {date}, ссылка - {link}'.format(date=pub_date.date(), link=a['href']))
            return pub_date, a['href']

loan_pub_date, loan_href=find_link(div_docs)

Найден файл с нужными данныи, дата публикации 2020-07-13, ссылка - /vfs/statistics/pdko/int_rat/loans_nonfin.xlsx


In [59]:
def convert_date(x):
    dct_month_transl={'Январь':1, 'Февраль':2, 'Март':3, 'Апрель':4, 'Май':5, 'Июнь':6, 
                  'Июль':7, 'Август':8, 'Сентябрь':9, 'Октябрь':10, 'Ноябрь':11, 'Декабрь':12}
    dtp=x.split(' ')
    return dt.date(int(dtp[1]), dct_month_transl[dtp[0]], 1)

pdf_loan=pd.read_excel(urljoin(strCBR_LOAN_SRC, loan_href), sheet_name='ставки_руб.', 
                       skiprows=4, usecols='A, F').dropna()

pdf_loan.columns=['date', 'loan_nonfin']
pdf_loan['date']=pdf_loan['date'].apply(convert_date)
pdf_loan.set_index('date', inplace=True)
pdf_loan

,loan_nonfin
date,
2014-01-01,9.15
2014-02-01,9.43
2014-03-01,10.29
2014-04-01,10.53
2014-05-01,10.60
...,...
2020-01-01,7.47
2020-02-01,7.47
2020-03-01,7.84


Сохраняем полученный фрейм в базе данных

In [63]:
_pdf=pds.DataFrameDATA(pdf_loan.copy())
_pdf.to_sql(strLOAN_table, con=conWork, if_exists='upsert', chunksize=int(1e4))

pds.DataFrameDATA([{'index':'date', 'col_name': '''Средневзвешенные процентные ставки по кредитам, 
предоставленным кредитными организациями нефинансовым организациям в рублях, до 1 года, 
включая "до востребования"'''}] ).set_index('index').to_sql(strLOAN_data_pass, con=conWork, if_exists='replace')

_=conWork.execute('''INSERT OR REPLACE INTO {tab_info}(utable, udate) VALUES
('{table_name}', '{update_date}') '''.format(tab_info=strInfoTalbe, 
                                          update_date=dt.datetime.now().strftime('%Y-%m-%d'), 
                                             table_name=strLOAN_table))

создание таблицы с описанием колонок loan и запроса выдачи данных

In [64]:
strLOAN_CreateView_LOAN='''
create view if not exists loan_agg as
select {loan_table}.date, {loan_table}.loan_nonfin
from {loan_table};
    '''.format(loan_table=strLOAN_table)

_=conWork.execute(strLOAN_CreateView_LOAN)

## Окончание работы: очистка и сжатие конечного файла SQLite3

In [65]:
# ВНИМАНИЕ!!! УДАЛЕНИЕ ФАЙЛА С ИНН ДЛЯ ЗАПРОСА БАЗЫ СПАРК!!!
try:
    remove(strFileINN)
except FileNotFoundError:
    pass
#==========================================================

import sqlite3
conn = sqlite3.connect(strBasePath, isolation_level=None)
conn.execute("VACUUM") # сжатие базы данных
conn.close()

strMD='''
<div class="alert alert-block alert-success">
<b>База данных - {bd_path} успешно обновлена<br>
<a href="{bankrupt_prob}?DATABASE={bd_path}" target="_blank">Перейти в Модель микроуровневого стресс-тестирования</a></b><br>
</div>
'''.format(bankrupt_prob='bankrupt_prob.ipynb', bd_path=strBasePath)
md(strMD)


<div class="alert alert-block alert-success">
<b>База данных - DB/bankrp.sqlite3 успешно обновлена<br>
<a href="bankrupt_prob.ipynb?DATABASE=DB/bankrp.sqlite3" target="_blank">Перейти в Модель микроуровневого стресс-тестирования</a></b><br>
</div>
